In [1]:
import numpy as np
import copy
import math
import itertools
import functools

In [2]:
def myLess(a,b):
    #First compare my classes
    if (isinstance(a, PointGroup)):
        if (a.dim < b.dim):
            return True
        if (b.dim < a.dim):
            return False
        if myLess(a.pre,b.pre):
            return True
        if myLess(b.pre, a.pre):
            return False
        return myLess(a.points,b.points)
    #Tuples only compared by the first value 
    if (type(a) is tuple):
        return myLess(a[0], b[0])
    #Not lists should have '<'
    if (type(a) is not list):
        return (a < b)
    #Lists are compared by the length
    if (len(a) < len(b)):
        return True
    if (len(a) > len(b)):
        return False
    #If lists have the same size - compare value by value
    for (x,y) in list(zip(a,b)):
        if myLess(x,y):
            return True
        if myLess(y,x):
            return False
    return False

def myIntLess(b,a):
    return myLess(a,b) - myLess(b,a)

def mySort(points):
    n = len(points)
    transp = 0
    for i in range(n):
        for j in range(n - 1 - i):
            if myLess(points[j + 1], points[j]):
                temp = points[j]
                points[j] = points[j + 1]
                points[j + 1] = temp
                if (isinstance(points[j], PointGroup)):
                    a = max(1, points[j].dim - 1)
                    b = max(1, points[j + 1].dim - 1)
                    transp +=  a * b
                else:
                    transp += 1 
    return transp

def Alt(points, rule = (0,)):
    if len(rule) == 1:
        head = points[:rule[0]]
        rest = list(points[rule[0]:])
        a = list(itertools.permutations(list(rest)))
        b = [1]
        n = len(rest)
        res = []
        for permutation in a:
            res.append(copy.deepcopy(head) + list(permutation))
        for i in range(2, n + 1):
            q = 1
            tmp = []
            for k in range(i):
                tmp = tmp + [q* x for x in b] 
                q *= -1
            b = tmp
        return res, b
    if len(rule) == 2:
        a, k_a = Alt(points[0], (rule[0],))
        b, k_b = Alt(points[1], rule[1:])
        res = []
        coef = []
        for (point_a,ind_a) in list(zip(a,k_a)):
            for (point_b,ind_b) in list(zip(b,k_b)):
                k = ind_a * ind_b
                res.append([point_a, point_b])
                coef.append(k)
        return res,coef
    
def Cyclic(points, rule = (0,)):
    if len(rule) == 1:
        head = points[:rule[0]]
        rest = list(points[rule[0]:])
        n = len(rest)
        res = []
        for i in range(len(rest)):
            res.append(copy.deepcopy(head) + list(rest[i:]) + list(rest[:i]))
        b = [1 - 2 * (x * (n + 1) % 2) for x in range(n)]
        return res, b
    if len(rule) == 2:
        a, k_a = Cyclic(points[0], (rule[0],))
        b, k_b = Cyclic(points[1], rule[1:])
        res = []
        coef = []
        for (point_a,ind_a) in list(zip(a,k_a)):
            for (point_b,ind_b) in list(zip(b,k_b)):
                k = ind_a * ind_b
                res.append([point_a, point_b])
                coef.append(k)
        return res,coef

In [3]:
class PointGroup:
    #def copr(self):
    #    pass
    def __repr__(self):
        pass       

class Delta(PointGroup):
    def __init__(self, points):
        self.total_dim = len(points) - 1
        self.dim = 0#len(points) - 1
        self.pre = []
        self.points = copy.deepcopy(points)
    
    def sort(self):
        mySort(self.points)
        return 1#0 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
   
    def phi(self):
        return [(Delta(copy.deepcopy(self.points)), 1)]
    def coprod(self):
        return [(Delta(copy.deepcopy(self.points)), 1)]
    def __repr__(self):
        return str('\Delta(' +', '.join(map(str,self.points))+ ')')
        
def reduction(tensor_sum):
    tensor_sum.sort(key=functools.cmp_to_key(myIntLess))
    n = 1
    while (n < len(tensor_sum)):
        if (not myLess(tensor_sum[n][0],tensor_sum[n-1][0]) and not myLess(tensor_sum[n-1][0],tensor_sum[n][0])):
            k = tensor_sum[n-1][1] + tensor_sum[n][1]
            tensor_sum[n-1] = tuple((tensor_sum[n-1][0], k))
            tensor_sum.pop(n)
        else:
            if (tensor_sum[n-1][1] == 0):
                tensor_sum.pop(n-1)
            else:
                n+=1
                
def list_splits(list_, n, k, pre = [], sign_ = 1):
    if n == 0:
        return [(copy.deepcopy(pre) + copy.deepcopy(list_), sign_)]
    if k == 0:
        sign = len(list_) * len(pre)
        sign = 1 - 2 * (sign % 2)
        return [(copy.deepcopy(list_) + copy.deepcopy(pre), sign_ * sign)]
    left = list_splits(list_[1:], n, k - 1, pre + [list_[0]], sign_)
    sign = 1 - 2 * (len(pre) % 2)
    right = list_splits(list_[1:], n - 1, k, [list_[0]] + pre, sign_ * sign)
    return left + right    

In [4]:
class AGroup(PointGroup):
    def __init__(self, prefix, points):
        self.total_dim = len(prefix) + len(points[0]) - 1
        self.dim = len(points[0]) - 1
        self.pre = list(prefix)
        self.points = copy.deepcopy(points)
    
    
    def sort(self):
        mySort(self.pre)
        k = mySort(self.points[0]) + mySort(self.points[1])
        return 1 -  2 * (k % 2)
    
    def simple_co(self, points, num):
        if (num == 1):
            # <pre + l_0 | l_rest, r_rest> x Delta(pre, l_0, r_rest)
            # <pre + m_0 | m_rest, l_rest> x Delta(pre, m_0, l_rest)
            a_l = self.pre[:] + [points[0][0]]
            a_r = self.pre[:] + [points[1][0]]
            mySort(a_l)
            mySort(a_r)
            
            b = list(points[0][1:])
            c = list(points[1][1:])
            
            k = 0
            k += mySort(b)
            k += mySort(c)
            k = 1 - 2 * (k % 2)
            
            left_delta = Delta(a_l[:] + c[:])
            mySort(left_delta.points)
            
            right_delta = Delta(a_r[:] + b[:])
            mySort(right_delta.points)
            
            left_agroup = AGroup(a_l[:], [b[:],c[:]])
            right_agroup = AGroup(a_r[:], [c[:],b[:]])
            
            l_r = num * (self.dim - num)
            l_r = 1 - 2 * (l_r % 2)
            return [
                ([left_delta, left_agroup], k),
                ([right_delta, right_agroup], k * l_r)
            ]
        
        else :
            # <pre + l_num | l_0  l_rest,r_0 r_rest> x <pre + r_res| l_0 l_num, r_0 r_num>
            # <pre + r_num | l_0  l_rest,r_0 r_rest> x <pre + l_res| l_0 l_num, r_0 r_num>
            # compute dim's
            
            num_ = num
            rest_ = self.dim - num 
            
            left_num = list(points[0])[1:num + 1]
            left_rest = list(points[0])[num + 1:]
            
            right_num = list(points[1])[1:num + 1]
            right_rest = list(points[1])[num + 1:]
            
            # addition to prefix
            a_l_num = self.pre[:] + left_num[:]
            a_r_num = self.pre[:] + right_num[:]
            a_l_rest = self.pre[:] + left_rest[:]
            a_r_rest = self.pre[:] + right_rest[:]
            
            l_0 = points[0][0]
            m_0 = points[1][0]
            
            
            #mySort(a_l_num)
            #mySort(a_r_num)
            #mySort(a_l_rest)
            #mySort(a_r_rest)
            
            A_l_num = AGroup(a_l_num,[[l_0] + left_rest[:], [m_0] + right_rest[:]])
            A_l_rest = AGroup(a_l_rest,[[l_0] + left_num[:], [m_0] + right_num[:]])
            A_r_num = AGroup(a_r_num,[[l_0] + left_rest[:], [m_0] + right_rest[:]])
            A_r_rest = AGroup(a_r_rest,[[l_0] + left_num[:], [m_0] + right_num[:]])
            
            k_l = A_l_num.sort() * A_r_rest.sort()
            
            k_r = A_r_num.sort() * A_l_rest.sort()
            
            k_r *= 1 - 2 *(num_ * rest_ % 2)
            
            #print(
            #([A_l_num, A_r_rest], k_l),
            #'\n',
            #([A_r_num, A_l_rest], k_r))
        return [
            ([A_l_num, A_r_rest], k_l),
            ([A_r_num, A_l_rest], -k_r),
        ]

    def coprod(self, num = 1):
        alt_points, signs = [], []
        res = []
        if num == 1:
            alt_points, signs = Cyclic(self.points, (0,0))
        else:
            alt_points, signs = Alt(self.points, (1,1))
        for points,sign_ in list(zip(alt_points, signs)):
            simple_res = self.simple_co(points, num)
            for elem,k in simple_res:
                res.append((elem, k * sign_))
        reduction(res)
        return res
    
    def simple_phi(self, points):
        if self.dim == 1:
            delta_ = Delta(self.pre[:] + [points[0][0],points[1][0]])
            mySort(delta_.points)
            return [(delta_, 1)]
        if self.dim == 2:
            g_group = GGroup(self.pre[:] + [points[0][0]],list(points[0])[1:] + list(points[1])[1:])
            #mySort(g_group.pre)
            #k = mySort(g_group.points)
            #k = 1 - 2 * (k % 2)
            k = g_group.sort()
            return [(g_group, k)]
        else:
            g_group = GGroup(self.pre[:] + [points[0][0]],list(points[0])[1:] + list(points[1])[1:])
            h_group = HGroup(self.pre[:] + [points[0][0], points[1][0]],[list(points[0])[1:],list(points[1])[1:]])
            k_g = g_group.sort()
            k_h = h_group.sort()
        return [
            (g_group, k_g), 
            (h_group, k_h)
        ]
        
    def phi(self):
        #alt_points, signs = Alt(copy.deepcopy(self.points), (0,0))
        alt_points, signs = Cyclic(copy.deepcopy(self.points), (0,0))
        res = []
        for points,sign_ in list(zip(alt_points, signs)):
            points_and_k = self.simple_phi(points)
            for point,k in  points_and_k:
                res.append((point, k * sign_))     
        reduction(res)
        return res
    
    def __repr__(self):
        return ('<' +', '.join(self.pre) +      '|'+   
                   ', '.join(self.points[0])+ ' ; '+
                   ', '.join(self.points[1])+ '>_{'+'A_{}'.format(self.dim)+'}')


In [5]:
class GGroup(PointGroup):
    def __init__(self, prefix, points):
        self.total_dim = len(prefix) + len(points) // 2
        self.dim = len(points) // 2
        self.pre = list(prefix)
        self.points = copy.deepcopy(points)
    
    def sort(self):
        mySort(self.pre)
        k = 0 if (self.dim == 2) else mySort(self.points)
        #k = mySort(self.points)
        return 1 -  2 * (k % 2)
    
    def simple_co(self, points, num = 1):
        res = []
        if num == 1: 
            half = len(points) // 2
            a_group = AGroup(self.pre[:],[list(points)[: half], list(points)[half:]])
            delta_ = Delta(self.pre[:] + list(points)[half:])
            #mySort(delta_.points)
            delta_.sort()
            a_phi = a_group.phi()
            for summand, k in a_phi:
                add = [copy.deepcopy(delta_), summand]
                q = mySort(add)
                q = 1- 2*(q%2)
                res.append((add, q * k))
        if num == 2:
            if self.dim == 4:
                g_1 = GGroup(
                    self.pre[:] + [points[0],points[1]],
                    [points[2],points[3],points[4],points[5]]
                )
                g_2 = GGroup(
                    self.pre[:] + [points[0],points[5]],
                    [points[1],points[2],points[6],points[7]]
                )
                dk = g_1.sort() * g_2.sort()
                res.append(([g_1, g_2], dk))
        reduction(res)
        return res
            
    def coprod(self, num = 1):
        alt_points = []
        signs = []
        if num > 1:
            pts,signs = Alt(self.points)
            alt_points = list(zip(pts, signs))
        else:
            alt_points = list_splits(self.points,self.dim, self.dim)
        res = []
        for points,sign_ in alt_points:
            simple_points = self.simple_co(points, num)
            for a_b, k in simple_points:
                res.append((a_b, k * sign_))
        reduction(res)
        return res
        
    def __repr__(self):
        return '\{'+ ', '.join(self.pre) + '|' + ', '.join(self.points)   + '\}_{'+'G_{}'.format(self.dim)+'}'

class HGroup(PointGroup):
    def __init__(self, prefix, points):
        self.total_dim = len(prefix) + len(points[0]) - 1
        self.dim = len(points[0]) - 1
        self.pre = list(prefix)
        self.points = copy.deepcopy(points)
    
    def sort(self):
        mySort(self.pre)
        k = mySort(self.points[0]) + mySort(self.points[1])
        return 1 -  2 * (k % 2)
    
    def simple_co(self, points, num = 1):
        if self.dim == 2:
            a_group_1 = AGroup(self.pre[:],[[points[0][0], points[1][1]], [points[0][1], points[1][0]]])
            a_group_2 = AGroup(self.pre[:],[[points[0][0], points[1][0]], [points[0][1], points[1][1]]])
            delta_ = a_group_1.phi() + a_group_2.phi()
            g_2 = GGroup(self.pre[:],[points[0][0], points[1][1], points[0][1], points[1][0]])
            q = mySort(g_2.points)
            q = 1 - 2 * (q % 2)
            reduction(delta_)
            res = []
            for element,k in delta_:
                res.append(([element, copy.deepcopy(g_2)], k * q))
            reduction(res)
            return res
        if self.dim == 3:
            if num == 1:
                res = []
                g_group = GGroup(
                    self.pre[:], 
                    points[0][0:3] + points[1][0:3]
                )
                
                #l_1 m_1 m_2
                delta_l_mm = Delta(self.pre[:] + [points[0][0],points[1][0],points[1][1]])
                #l_1 l_2 m_1
                delta_ll_m = Delta(self.pre[:] + [points[0][0],points[0][1],points[1][0]])
                
                # l_3|l_2,m_3,m_4; l_1,m_1,m_2 *
                h_l_m = HGroup(self.pre[:] + [points[0][2]], 
                              [
                                  [points[0][1],points[1][2],points[1][3]],
                                  [points[0][0],points[1][0],points[1][1]]
                              ])
                
                # m_3|l_2,l_3,m_4; l_1,m_1,m_2 *
                h_m_m = HGroup(self.pre[:] + [points[1][2]], 
                              [
                                  [points[0][1],points[0][2],points[1][3]],
                                  [points[0][0],points[1][0],points[1][1]]
                              ])
                # l_3|l_4,m_2,m_3; l_1,l_2,m_1
                h_l_l = HGroup(self.pre[:] + [points[0][2]], 
                              [
                                  [points[0][3],points[1][1],points[1][2]],
                                  [points[0][0],points[0][1],points[1][0]]
                              ])
                
                # m_3|l_3,l_4,m_2; l_1,l_2,m_1
                h_m_l = HGroup(self.pre[:] + [points[1][2]], 
                              [
                                  [points[0][2],points[0][3],points[1][1]],
                                  [points[0][0],points[0][1],points[1][0]]
                              ])
                delta_ll_m.sort()
                delta_l_mm.sort()
                k_l_m = h_l_m.sort()
                k_l_l = h_l_l.sort()
                k_m_m = h_m_m.sort()
                k_m_l = h_m_l.sort()
                k_g = g_group.sort()
                res.append(([copy.deepcopy(delta_ll_m), copy.deepcopy(g_group)], k_g,)) #!!!!!!!!!!!!!!!!!!!!
                res.append(([copy.deepcopy(delta_l_mm), copy.deepcopy(g_group)], k_g,)) #!!!!!!!!!!!!!!!!!!!!
                res.append(([copy.deepcopy(delta_l_mm), h_l_m],  k_l_m,))
                res.append(([copy.deepcopy(delta_l_mm), h_m_m], - k_m_m,))
                res.append(([copy.deepcopy(delta_ll_m), h_l_l], k_l_l,))
                res.append(([copy.deepcopy(delta_ll_m), h_m_l], -k_m_l,))
                return res
            if num == 2:
                g_2_1 = GGroup(self.pre[:] + [points[0][0]],[points[1][0]] + points[0][1:])
                g_2_2 = GGroup(self.pre[:] + [points[1][0]],[points[0][0]] + points[1][1:])
                dg = g_2_1.sort()
                dg *= g_2_2.sort()
                base = [g_2_1, g_2_2]
                dg *= (1 - 2 * mySort(base))
                return [(base, dg)]
        
    def coprod(self, num = 1):
        alt_points, signs = [], []
        alt_points, signs = Alt(self.points, (0,0))
        res = []
        for points,sign_ in list(zip(alt_points, signs)):
            simple_points = self.simple_co(points, num)
            for (a_b, k) in simple_points:
                dk = 1 #0.25 if (self.dim == 2) else 1
                res.append((a_b, dk * k * sign_))
        reduction(res)
        return res
        
    def __repr__(self):
        return '['+', '.join(self.pre)+'|' + ', '.join(self.points[0])+ ' ; ' + ', '.join(self.points[1])+ ']_{' + 'H_{}'.format(self.dim)+'}'

In [6]:
def take_phi(tensors):
    res = []
    for tensor in tensors:
        k = tensor[1]
        product = copy.deepcopy(tensor[0])
        phi_ = [factor.phi() for factor in product]
        n = 1
        for list_ in phi_:
            n *= len(list_)
        for ind_ in range(n):
            tmp = []
            tmp_k = k
            for i in range(len(product)):
                j = ind_ % len(phi_[i])
                ind_ = ind_ // len(phi_[i])
                tmp_k *= phi_[i][j][1]
                tmp.append(copy.deepcopy(phi_[i][j][0]))
            q = mySort(tmp)
            tmp_k *= (1 - 2 *(q % 2))
            res.append((tmp, tmp_k))
    reduction(res)
    return res

def take_coprod(tensors, num = 1):
    res = []
    for tensor in tensors:
        k = tensor[1]
        product = copy.deepcopy(tensor[0])
        dk = -1
        for ind_ in range(len(product)):
            dk *= -1
            if (product[ind_].dim <= num):
                continue
            tmp = copy.deepcopy(product)
            head = tmp[:ind_]
            tail = tmp[ind_ + 1:]
            update = tmp[ind_].coprod(num)
            for factor,fk in update:
                tmp_k = k * fk
                add_term = copy.deepcopy(head) + copy.deepcopy(factor) + copy.deepcopy(tail)
                q = mySort(add_term)
                tmp_k *= (1 - 2 *(q % 2))
                res.append((add_term, tmp_k * dk))
    reduction(res)
    return res

def myMult(tensors, scalar):
    res = []
    for tensor in tensors:
        k = tensor[1]
        item = copy.deepcopy(tensor[0])
        res.append((item, int(scalar * k)))
    return res


In [7]:
def basis_separation(tensor_sum):
    num = 0
    while (type(tensor_sum[0][0][num]) is Delta):
        num += 1
        if num >= len(tensor_sum[0][0]):
            return tensor_sum
    res = []
    tmp = []
    basic_element = copy.deepcopy(tensor_sum[0][0][:num])
    for tensor in tensor_sum:
        if myIntLess(basic_element, tensor[0][:num]) == 0:
            tmp.append((copy.deepcopy(tensor[0][num:]), tensor[1]))
        else:
            res.append((basic_element, tmp))
            tmp = [(copy.deepcopy(tensor[0][num:]), tensor[1])]
            basic_element = copy.deepcopy(tensor[0][:num])           
    tmp.append((basic_element, tmp))
    return res

def extracting_pattern(tensor_sum, rule = (1,1)):
    res = []
    for (x,k) in tensor_sum:
        k = abs(k)
        y = pattern(x, rule)
        res.append((y,1))
    reduction(res)
    return res